In [1]:
import math
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# Step 1 : Dealing with missing values

In [2]:
# TODO Replace with your own path to the csv file before running
path_to_original_csv = "../Datasets/DOB_Job_Application_Filings.csv"

jobs_df = pd.read_csv(path_to_original_csv)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (5,6,13,14,21,32,38,59,60,68,78,79) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
jobs_df.head()
# jobs_df.info()

,Job #,Doc #,Borough,House #,Street Name,Block,Lot,Bin #,Job Type,Job Status,...,SPECIAL_ACTION_STATUS,SPECIAL_ACTION_DATE,BUILDING_CLASS,JOB_NO_GOOD_COUNT,GIS_LATITUDE,GIS_LONGITUDE,GIS_COUNCIL_DISTRICT,GIS_CENSUS_TRACT,GIS_NTA_NAME,GIS_BIN
0,110083524,2,MANHATTAN,152,WEST 57 STREET,1009,5,1084455,A2,X,...,A,NaN,NaN,0,40.765185,-73.979279,4.0,137.0,Midtown-Midtown South,1084455.0
1,110083533,1,MANHATTAN,444,EAST 52 STREET,1363,27,1040141,A2,X,...,N,NaN,NaN,0,40.754694,-73.964242,4.0,8603.0,Turtle Bay-East Midtown,1040141.0
2,110083542,1,MANHATTAN,2,WARDS ISLAND,1819,10,1085639,A1,J,...,W,08/25/2008,NaN,0,40.783512,-73.930157,8.0,240.0,park-cemetery-etc-Manhattan,NaN
3,110083542,2,MANHATTAN,2,WARDS ISLAND,1819,10,1085639,A1,J,...,W,08/25/2008,NaN,0,40.783512,-73.930157,8.0,240.0,park-cemetery-etc-Manhattan,NaN
4,110083542,3,MANHATTAN,2,WARDS ISLAND,1819,10,1085639,A1,D,...,W,08/25/2008,NaN,0,40.783512,-73.930157,8.0,240.0,park-cemetery-etc-Manhattan,NaN


In [4]:
rem = 0
print("Number of rows before cleaning    : {}".format(jobs_df.shape[0]))
print("Number of columns before cleaning : {}".format(jobs_df.shape[1]))
for col in jobs_df.columns:
    if jobs_df[col].isna().sum() >0:
        rem += 1
        print("Column name, type : {}, {} : values(s) missing".format(col, jobs_df[col].dtypes, jobs_df[col].isna().sum()))
print("Remaining columns to clean : {}".format(rem))

Number of rows before cleaning    : 1756231
Number of columns before cleaning : 96
Column name, type : House #, object : values(s) missing
Column name, type : Street Name, object : values(s) missing
Column name, type : Block, object : values(s) missing
Column name, type : Lot, object : values(s) missing
Column name, type : Latest Action Date, object : values(s) missing
Column name, type : Community - Board, object : values(s) missing
Column name, type : Cluster, object : values(s) missing
Column name, type : Landmarked, object : values(s) missing
Column name, type : Adult Estab, object : values(s) missing
Column name, type : Loft Board, object : values(s) missing
Column name, type : City Owned, object : values(s) missing
Column name, type : Little e, object : values(s) missing
Column name, type : PC Filed, object : values(s) missing
Column name, type : eFiling Filed, object : values(s) missing
Column name, type : Plumbing, object : values(s) missing
Column name, type : Mechanical, obje

In [5]:
# Cleaning Job types columns :
job_types = ["Plumbing", "Mechanical", "Boiler", "Fuel Burning", "Fuel Storage", "Standpipe", 
             "Sprinkler", "Fire Alarm", "Equipment", "Fire Suppression", "Curb Cut", "Other"]

for job_type in job_types:
    jobs_df[job_type] = jobs_df[job_type].map(lambda val: 1 if val == "X" else 0)

In [6]:
# Cleaning Binary variables
jobs_df["City Owned"] = jobs_df["City Owned"].map(lambda val: 1 if val == "Y" else 0)
jobs_df["Landmarked"] = jobs_df["Landmarked"].map(lambda val: 1 if val == "Y" else 0)
jobs_df["Adult Estab"] = jobs_df["Adult Estab"].map(lambda val: 0 if val == "N" else 1)
jobs_df["Little e"] = jobs_df["Little e"].map(lambda val: 1 if val == "H" else 0)

jobs_df["Zoning Dist1"] = jobs_df["Zoning Dist1"].map(lambda val: 0 if pd.isnull(val)  else 1) # Residence
jobs_df["Zoning Dist2"] = jobs_df["Zoning Dist2"].map(lambda val: 0 if pd.isnull(val)  else 1) # Commercial
jobs_df["Zoning Dist3"] = jobs_df["Zoning Dist3"].map(lambda val: 0 if pd.isnull(val)  else 1) # Manufacturing

jobs_df["Special District 1"] = jobs_df["Special District 1"].map(lambda val: 1 if not pd.isnull(val) else 0)
jobs_df["Special District 2"] = jobs_df["Special District 2"].map(lambda val: 1 if not pd.isnull(val) else 0)
jobs_df["Special District"] = jobs_df["Special District 1"] + jobs_df["Special District 2"]
jobs_df["Special District"] = jobs_df["Special District"].map(lambda val: 0 if val == 0 else 1)

# Cleaning missing values in other columns
jobs_df["Vertical Enlrgmt"] = jobs_df["Vertical Enlrgmt"].map(lambda val: 0 if pd.isnull(val) or val == "N" else 1)
jobs_df["Horizontal Enlrgmt"] = jobs_df["Horizontal Enlrgmt"].map(lambda val: 0 if pd.isnull(val) or val == "N" else 1)
jobs_df["Other Description"] = jobs_df["Other Description"].map(lambda desc: desc if not pd.isnull(desc) else "")

In [7]:
# Dropping uniteresting columns
useless_columns = ["PC Filed", "eFiling Filed", "Cluster", "Loft Board", "Non-Profit", "Other Description",
                   "Applicant's First Name", "Applicant's Last Name", "Applicant Professional Title", "Applicant License #",
                   "Professional Cert", "Professional Cert", "Owner's First Name", "Owner's Last Name", "Owner's Business Name",
                   "Owner's House Number", "Owner'sHouse Street Name","Existing Dwelling Units", "Proposed Dwelling Units",
                   "City ", "State", "Zip", "Owner'sPhone #", "BUILDING_CLASS", "Job Description", "SIGNOFF_DATE", "Assigned",
                   "Approved", "Fully Permitted", "Existing Occupancy", "Proposed Occupancy", "SPECIAL_ACTION_DATE", 
                   "SPECIAL_ACTION_STATUS", "Site Fill", "Special District 1", "Special District 2", "DOBRunDate"]

print(len(useless_columns))

jobs_df = jobs_df.drop(labels = useless_columns, axis = 1)

37


In [8]:
# Deletes rows with na values on these columns:
columns_drop_na = ["House #", "Street Name", "Block", "Lot", "GIS_LATITUDE", "GIS_LONGITUDE",
                   "GIS_COUNCIL_DISTRICT", "GIS_CENSUS_TRACT", "GIS_NTA_NAME", "GIS_BIN", "Latest Action Date",
                   "Community - Board", "Paid", "Fully Paid", "Owner Type"]

jobs_df = jobs_df.dropna(subset = columns_drop_na)

In [9]:
# Removing the rows for which we don't know the initial cost
jobs_df = jobs_df[jobs_df['Initial Cost']!="$0.00"]

In [10]:
# Converting prices to floats
jobs_df["Initial Cost"] = jobs_df["Initial Cost"].apply(lambda x: x.replace('$','')).apply(lambda x: x.replace(',','')).astype(np.float64)
jobs_df["Total Est. Fee"] = jobs_df["Total Est. Fee"].apply(lambda x: x.replace('$','')).apply(lambda x: x.replace(',','')).astype(np.float64)

In [11]:
# Dates to datetime format
dates_columns = ["Paid", "Fully Paid","Pre- Filing Date", "Latest Action Date"]

for c in dates_columns : 
    jobs_df[c] = pd.to_datetime(jobs_df[c])

In [12]:
# # Horizontal Enlrgmt to dummy variables
# l = list(data["Horizontal Enlrgmt"])
# for i in range(len(l)):
#     if l[i]=='Y':
#         l[i]=1
#     elif l[i]=='N':
#         l[i]=0
#     elif l[i]=='0':
#         l[i]=0
# data["Horizontal Enlrgmt"] = pd.Series(l)

In [13]:
rem = 0
print("Number of rows after cleaning    : {}".format(jobs_df.shape[0]))
print("Number of columns after cleaning : {}".format(jobs_df.shape[1]))
for col in jobs_df.columns:
    if jobs_df[col].isna().sum() >0:
        rem += 1
        print("Column name, type : {}, {} : values(s) missing".format(col, jobs_df[col].dtypes, jobs_df[col].isna().sum()))


Number of rows after cleaning    : 1222083
Number of columns after cleaning : 61


# Step 2 : Adjusting prices for inflation

In [17]:
# Loading the buildings metadata
cpi_path = "../Datasets/CPI.csv"
cpi_df = pd.read_csv(cpi_path)

In [29]:
cpi_df.loc[252,"Value"]

257.971

In [24]:
cpi_df

,Series ID,Year,Period,Label,Value
0,CUUR0000SA0,1999,M01,1999 Jan,164.300
1,CUUR0000SA0,1999,M02,1999 Feb,164.500
2,CUUR0000SA0,1999,M03,1999 Mar,165.000
3,CUUR0000SA0,1999,M04,1999 Apr,166.200
4,CUUR0000SA0,1999,M05,1999 May,166.200
...,...,...,...,...,...
257,CUUR0000SA0,2020,M06,2020 Jun,257.797
258,CUUR0000SA0,2020,M07,2020 Jul,259.101
259,CUUR0000SA0,2020,M08,2020 Aug,259.918
260,CUUR0000SA0,2020,M09,2020 Sep,260.280


# Step 3 : Merging with the building info dataset

In [14]:
# Loading the buildings metadata
building_path = "../Datasets/pluto_20v6.csv"
buildings_df = pd.read_csv(building_path)

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (19,20,22,24) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [15]:
buildings_df = buildings_df.filter(items = ['address',"bldgclass","landuse","bldgarea","numfloors","unitsres","unitstotal","bldgfront","bldgdepth","lottype","assesstot","yearbuilt","yearalter1","yearalter2","firm07_flag","lot",'block','ct2010'])
buildings_df['firm07_flag'] = buildings_df['firm07_flag'].fillna(0)


In [18]:
# Merging the cleaned jobs dataset with the buildings metadata
final_df = pd.merge(buildings_df, jobs_df, how = "right", left_on = ['ct2010','lot','block'], right_on = ['GIS_CENSUS_TRACT','Lot','Block'])

# Dropping rows with missing values
final_df = final_df.dropna()

final_df = final_df.loc[:, final_df.columns != 'Unnamed: 0']

In [19]:
output_path = "../Datasets/Jobs_and_buildings_merged_v2.0.csv"
final_df.to_csv(output_path)